<h1><center>Build HowsMyNHS website</center></h1>

This is the main script used to automatically generate the html for <a href="https://howsmynhs.co.uk/">HowsMyNHS</a>.

In [1]:
import build_website as bw
data = 'NHSwaiting.npy'

<h2>Generate Plots</h2>

In [3]:
bw.plotWaitingData(data)

Generating 4 hour waiting time graphs... Done.


In [4]:
engWaiting = movingAverage(waiting[0,:][waiting[0,:] != '-'])

<h2>Make homepage</h2>

In [2]:
bw.MakeHomepage(data)

Building homepage... Done


<h2>Make all the other pages</h2>

In [8]:
headHTML = '''
<html>
<head>

<!-- Global site tag (gtag.js) - Google Analytics -->
<script async src="https://www.googletagmanager.com/gtag/js?id=UA-154345093-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-154345093-1');
</script>


<title>How's my NHS?</title>
<link rel="stylesheet" type="text/css" href="../style.css">

</head>
<body>
<div class="maintitle">
<h1><center><a href="../index.html"><img src="../logo.png" alt="How's my NHS?" /></a></center></h1>
</div>'''

tailHTML = '''
</body>
</html>'''

whatNextHTML = '''
    </br>
    <div class ="box">
    <h2 class = \"subtitle\"><center>What Next?</center></h2>\n
    
    <p><b style="color:#005EB8;">Prevent the spread:</b> With the outbreak of the Coronavirus (COVID-19)
    the NHS is under more pressure than ever. We <b>need</b> to do everything we can to slow its
    spread. Follow the official government guidance: 
    <ul>
    <li>Stay at home as much as possible</li>
    <li>Work from home if you can</li>
    <li>Limit contact with other people</li>
    <li>Keep your distance from people not in your household (2 metres apart where possible)</li>
    <li>Wash your hands regularly</li>
    
    </ul>
    Many young people experience few to no symptoms at all but can still carry the virus and pass 
    it on to those who are more vulnerable.
    </p>
    <p>
    Stay informed with reliable sources such as the governments 
    <a href="https://www.gov.uk/coronavirus" target="_blank">offical Coronavirus page</a>.
    </p> 

    <p><b style="color:#005EB8;">Share the message:</b> Fight for change. Let your friends and family know that the Conservatives 
    are not the party for the NHS. One vote may have little impact, particularly with our broken electoral system. 
    But together we can bring about great change.</p> 

<div id="share-buttons" align="center" style="color:#ffffff;">
    
    <!-- Twitter -->
    <a href="https://twitter.com/share?url=https://howsmynhs.co.uk/&amp;text=How%20has%20YOUR%20NHS%20changed%20under%20the%20Tories?&amp;hashtags=NHS"target="_blank">
    <img src="https://simplesharebuttons.com/images/somacro/twitter.png" alt="Twitter" /></a>
    
    <!-- Email -->
    <a href="mailto:?Subject=Simple Share Buttons&amp;Body=How%20has%20YOUR%20NHS%20changed%20under%20the%20Tories?%20 https://howsmynhs.co.uk/">
    <img src="https://simplesharebuttons.com/images/somacro/email.png" alt="Email" /></a>
 
    <!-- Facebook -->
    <a href="http://www.facebook.com/sharer.php?u=https://howsmynhs.co.uk/" target="_blank">
    <img src="https://simplesharebuttons.com/images/somacro/facebook.png" alt="Facebook" /></a>
    
    <!-- Reddit -->
    <a href="http://reddit.com/submit?url=https://howsmynhs.co.uk/&amp;title=How has your NHS changed under the Tories?" target="_blank">
    <img src="https://simplesharebuttons.com/images/somacro/reddit.png" alt="Reddit" /></a>

    <!-- Tumblr-->
    <a href="http://www.tumblr.com/share/link?url=https://howsmynhs.co.uk/.com&amp;title=Simple Share Buttons" target="_blank">
    <img src="https://simplesharebuttons.com/images/somacro/tumblr.png" alt="Tumblr" /></a>
    
    <!-- LinkedIn -->
    <a href="http://www.linkedin.com/shareArticle?mini=true&amp;url=https://howsmynhs.co.uk/" target="_blank">
    <img src="https://simplesharebuttons.com/images/somacro/linkedin.png" alt="LinkedIn" /></a>
    
   <! -- Share buttons from: https://simplesharebuttons.com/html-share-buttons/ --!>
</div>

    <p><b style="color:#005EB8;">Email your MP:</b> Enter your postcode 
    <a href="https://members.parliament.uk/" target="_blank">here</a>.
    to find the email address for your local MP. Send them this graph of 
    how <b>your</b> local A&E has changed and urge them to use their power to
    protect it from further damage and to restore our NHS to it's former 
    glory.
    </p>

    <p><b style="color:#005EB8;">Use this data:</b> All of the data presented here, as well as the code used to collect it, is <a href="https://github.com/David-Ellis/NHSdata" target="_blank"> freely availble via github</a>. 
    You don't need to cite us. This is all publically available data from <a href="https://www.england.nhs.uk/statistics/statistical-work-areas/ae-waiting-times-and-activity/" target="_blank">NHS England</a>. 
    </p>    
    
    </br></br></br></br></br></br></br></br>
    '''

#<a href="https://twitter.com/intent/tweet?screen_name=BorisJohnson&ref_src=twsrc%5Etfw" 
#class="twitter-mention-button" data-show-count="false">Tweet to @BorisJohnson</a>
#<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

#<a href="https://twitter.com/howsmynhs?ref_src=twsrc%5Etfw"
#class="twitter-follow-button" data-show-count="false">Follow @HowsMyNHS</a>
#<script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

In [13]:
hospitalLinksList = []

attendance = NHSdata[2]
waiting = NHSdata[3]
   
for i, name in enumerate(names):
    mask = (waiting[i,:] != '-')
    
    pageExist = False
    if sum(mask)>12:
        pageExist = True
        
    print(i,name, pageExist)
    
    if i == 0 and pageExist:
        url_prefix = '_'.join(name.lower().split(' '))
        url = ''.join([url_prefix,".html"])
        file = open("hospitals/{}".format(url), "w")
        figName = '_'.join(name.lower().split(' '))
        path = "../figures/{}.png".format(figName)
        #print("figures/{}.png".format(figName))
        subTitleHTML = '''<div class = \"box\">
        \n<h2 class = \"subtitle\"><center>All of England</center></h2>\n'''
        
        imgHTML = "<center><img src=\"{}\" alt=\"{}\"></center>".format(path, name)
        
        supTextHTML = u'''
            
            <p>After nearly a decade of Conservative rule, in December over <b>500,000</b> more people were made to wait
            <b>over four hours</b> to be seen at A&E than in 2011. That's an increase of over <b>900%</b>.
            The number of people attending A&E, however, had only increased by less than 30%.<p>
            
            {}
            
            <p>Now, Brexit and a trade deal with Trump's US threatens to increase the NHS drug bill from &pound18 billion to as much 
            as <b>&pound45 billion</b> a year while shutting out the nurses, carers and other workers that the health service depends on.</p>
            
            <p>Over the last few months this number has decreased dramatically. However, this is because the number of people attending
            A&E has fallen by more than 50% as people choose to stay at home to help reduce the pressure on the NHS 
            during the current pandemic.</p>            
            
            \n</div>'''.format(imgHTML)
        
        
        file.write(''.join([headHTML,subTitleHTML,supTextHTML,whatNextHTML,tailHTML]))
        file.close() 
    elif sum(attendance[i,:] != '-')>=12 and pageExist:
        url_prefix = '_'.join(name.lower().split(' '))
        url = ''.join([url_prefix,".html"])
        file = open("hospitals/{}".format(url), "w")
        figName = '_'.join(name.lower().split(' '))
        path = "../figures/{}.png".format(figName)
        #print("figures/{}.png".format(figName))
        subTitleHTML = '''\n<div class = \"box\">
        <h2 class = \"subtitle\"><center>{}</center></h2>\n'''.format(name)
        
        imgHTML = "<center><img src=\"{}\" alt=\"{}\"></center>".format(path, name)
        
        smoothWait = movingAverage(waiting[i,:][waiting[i,:] != '-'])
        avAtt = np.mean(attendance[i,:][attendance[i,:] != '-'])
        sampleDates = dates[waiting[i,:] != '-']
        if max(smoothWait) < 15 and avAtt<2000 and pageExist:
            supTextHTML = '''
            <!--Minimal Change Hospital + less than 2000 monthy attendance-->
            
            <p>It looks like things haven't changed much for your hospital! On average only {} people attend this A&E 
            each month.</p>
            <p>Unfortunately, things aren't so great for the rest of England. After nearly a decade of Conservative rule,
            each month over <b>400,000</b> more people are made to wait <b>over four hours</b> to be seen at A&E than 
            in {}.
            
            {}
            
            <p>Now, Brexit and a trade deal with Trump's US threatens to increase the NHS drug bill from &pound18 billion to as much 
            as <b>&pound45 billion</b> a year while shutting out the nurses, carers and other workers that the health service depends on.</p>
            
            <p>Note: Over the past three months A&E attendance across England has fallen by more than 50% as people choose to stay at home to help reduce the pressure on the NHS 
            during the current pandemic.</p>
            
            \n</div>'''.format(int(avAtt), int(round(min(sampleDates))), imgHTML)
            
        elif max(smoothWait) < 15 and pageExist:
            supTextHTML = '''
            <!--Minimal Change Hospital + more than 2000 monthy attendance-->
            
            <p> It looks like things haven't changed much for your hospital!</p>
            <p>Unfortunately, things aren't so great for the rest of England. After nearly a decade of Conservative rule,
            each month over <b>400,000</b> more people are made to wait <b>over four hours</b> to be seen at A&E than in 2011.
            
            {}
            
            <p>Now, Brexit and a trade deal with Trump's US threatens to increase the NHS drug bill from &pound18 billion to as much 
            as <b>&pound45 billion</b> a year while shutting out the nurses, carers and other workers that the health service depends on.</p>
            
            <p>Note: Over the past three months A&E attendance across England has fallen by more than 50% as people choose to stay at home to help reduce the pressure on the NHS 
            during the current pandemic.</p>
            \n</div>'''.format(imgHTML)
            
        elif avAtt>2000 and smoothWait[0]-smoothWait[-1]>100 and pageExist:
            diff = smoothWait[0]-smoothWait[-1]
            supTextHTML = '''
            <p>After nearly a decade of Conservative rule, on average, <b>{}</b> more people are being left to wait over 
            four hours at A&E at <b>your</b> hospital than back in {}.</p>
            
            {}
            
            <p>And things are bad for the rest of England too. Each month over <b>400,000</b> more people are made to wait
            <b>over four hours</b> to be seen at A&E than in 2011.
            
            <p>Now, Brexit and a trade deal with Trump's US threatens to increase the NHS drug bill from &pound18 billion to as much 
            as <b>&pound45 billion</b> a year while shutting out the nurses, carers and other workers that the health service depends on.</p>

            <p>Note: Over the past three months A&E attendance across England has fallen by more than 50% as people choose to stay at home to help reduce the pressure on the NHS 
            during the current pandemic.</p>
            \n</div>'''.format(int(diff),int(np.floor(min(sampleDates))), imgHTML)
        elif pageExist:
            #print(name)
            supTextHTML = '''
            <p> The data show's that thing's have gotten worse in your hospital. With {} more people each month 
             being forced to wait over 4 hours to be seen at A&E since {}. It many English hospitals, the situation is much
             worse. After nearly a decade of Conservative rule, each month over <b>400,000</b> more people are made 
             to wait <b>over four hours</b> to be seen at A&E than in 2011.</p>
             
             {}
             
            <p>Now, Brexit and a trade deal with Trump's US threatens to increase the NHS drug bill from &pound18 billion to as much 
            as <b>&pound45 billion</b> a year while shutting out the nurses, carers and other workers that the health service depends on.</p>
            
            <p>Note: Over the past three months A&E attendance across England has fallen by more than 50% as people choose to stay at home to help reduce the pressure on the NHS 
            during the current pandemic.</p>
            \n</div>
            '''.format(int(diff), int(np.floor(min(sampleDates))), imgHTML)
        

        
        file.write(''.join([headHTML,subTitleHTML,supTextHTML,whatNextHTML,tailHTML]))
        file.close() 

0 England True
1 Bedfordshire Hospitals NHS Foundation Trust False
2 Cambridge University Hospitals NHS Foundation Trust True
3 Clacton Hospital True
4 East And North Hertfordshire NHS Trust True
5 East Suffolk And North Essex NHS Foundation Trust True
6 Fryatt Hospital True
7 Hertfordshire Community NHS Trust True
8 Herts Urgent Care (Ascots Lane) True
9 James Paget University Hospitals NHS Foundation Trust True
10 Mid And South Essex NHS Foundation Trust False
11 Milton Keynes University Hospital NHS Foundation Trust True
12 Norfolk And Norwich University Hospitals NHS Foundation Trust True
13 North West Anglia NHS Foundation Trust True
14 Putnoe Medical Centre Walk In Centre True
15 The Princess Alexandra Hospital NHS Trust True
16 The Queen Elizabeth Hospital, King's Lynn, NHS Foundation Trust True
17 West Hertfordshire Hospitals NHS Trust True
18 West Suffolk NHS Foundation Trust True
19 Barking, Havering And Redbridge University Hospitals NHS Trust True
20 Barts Health NHS Trust 

253 nan False
254 nan False
255 nan False
256 nan False
257 nan False
258 nan False
259 nan False
260 nan False
261 nan False
262 nan False
263 nan False
264 nan False
265 nan False
266 nan False
267 nan False
268 nan False
269 nan False
270 nan False
271 nan False
272 nan False
273 nan False
274 nan False
275 nan False
276 nan False
277 nan False
278 nan False
279 nan False
280 nan False
281 nan False
282 nan False
283 nan False
284 nan False
285 nan False
286 nan False
287 nan False
288 nan False
289 nan False
290 nan False
291 nan False
292 nan False
293 nan False
294 nan False
295 nan False
296 nan False
297 nan False
298 nan False
299 nan False
300 nan False
301 nan False
302 nan False
303 nan False
304 nan False
305 nan False
306 nan False
307 nan False
308 nan False
309 nan False
310 nan False
311 nan False
312 nan False
313 nan False
314 nan False
315 nan False
316 nan False
317 nan False
318 nan False
319 nan False
320 nan False
321 nan False
322 nan False
323 nan False
324 na